In [2]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import copy
import pandas as pd
import time
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from utils.subspace_clustering_helper_funcs import *
from utils.preprocessing import *

In [3]:
#file_path_kai = 'C:\\Users\\kdmen\\Desktop\\Research\\Data\\$M\\Filtered_Datasets\\metadata_IMU_EMG_allgestures_allusers.pkl'
file_path_brc = 'D:\\Kai_MetaGestureClustering_24\\saved_datasets\\Filtered_Datasets\\metadata_IMU_EMG_allgestures_allusers.pkl'
file_path_ben = "C:\\Users\\rubin\\Research\\metadata_IMU_EMG_allgestures_allusers.pkl"

print("Loading")
start_time = time.time()
data_df = pd.read_pickle(file_path_ben)
end_time = time.time()
print(f"Completed in {end_time - start_time}s")

Loading
Completed in 0.08789587020874023s


In [4]:
print(data_df.shape)
data_df.head()

(204800, 91)


,Participant,Gesture_ID,Gesture_Num,IMU1_ax,IMU1_ay,IMU1_az,IMU1_vx,IMU1_vy,IMU1_vz,IMU2_ax,...,EMG7,EMG8,EMG9,EMG10,EMG11,EMG12,EMG13,EMG14,EMG15,EMG16
0,P102,pan,1,0.341797,-0.939941,0.000977,-0.007450,-0.192625,0.005321,-0.380859,...,0.000002,0.000002,0.000003,0.000020,0.000004,0.000004,0.000002,0.000009,0.000001,0.000002
1,P102,pan,1,0.336178,-0.963185,0.003898,0.009595,-0.190446,-0.026116,-0.394547,...,0.000003,0.000003,0.000003,0.000014,0.000007,0.000007,0.000002,0.000017,0.000001,0.000002
2,P102,pan,1,0.353539,-0.963704,0.011711,0.095966,-0.205480,-0.155563,-0.398406,...,0.000003,0.000003,0.000004,0.000007,0.000004,0.000005,0.000003,0.000020,0.000003,0.000002
3,P102,pan,1,0.352841,-0.950288,0.011509,0.058836,-0.184871,-0.083567,-0.389230,...,0.000003,0.000003,0.000006,0.000005,0.000004,0.000003,0.000004,0.000015,0.000003,0.000003
4,P102,pan,1,0.372621,-0.991273,0.029847,0.293946,-0.178756,-0.281361,-0.396043,...,0.000003,0.000002,0.000008,0.000003,0.000007,0.000022,0.000004,0.000017,0.000002,0.000003


## Filter out P111 P124 and P131 and unimpared participants

In [5]:
filtered_df = data_df
filtered_df = data_df[~data_df['Participant'].isin(['P004', 'P005', 'P006', 'P008', 'P010', 'P011', 'P111','P124','P131'])]
print(filtered_df.shape)
filtered_df.head()

(147200, 91)


,Participant,Gesture_ID,Gesture_Num,IMU1_ax,IMU1_ay,IMU1_az,IMU1_vx,IMU1_vy,IMU1_vz,IMU2_ax,...,EMG7,EMG8,EMG9,EMG10,EMG11,EMG12,EMG13,EMG14,EMG15,EMG16
0,P102,pan,1,0.341797,-0.939941,0.000977,-0.007450,-0.192625,0.005321,-0.380859,...,0.000002,0.000002,0.000003,0.000020,0.000004,0.000004,0.000002,0.000009,0.000001,0.000002
1,P102,pan,1,0.336178,-0.963185,0.003898,0.009595,-0.190446,-0.026116,-0.394547,...,0.000003,0.000003,0.000003,0.000014,0.000007,0.000007,0.000002,0.000017,0.000001,0.000002
2,P102,pan,1,0.353539,-0.963704,0.011711,0.095966,-0.205480,-0.155563,-0.398406,...,0.000003,0.000003,0.000004,0.000007,0.000004,0.000005,0.000003,0.000020,0.000003,0.000002
3,P102,pan,1,0.352841,-0.950288,0.011509,0.058836,-0.184871,-0.083567,-0.389230,...,0.000003,0.000003,0.000006,0.000005,0.000004,0.000003,0.000004,0.000015,0.000003,0.000003
4,P102,pan,1,0.372621,-0.991273,0.029847,0.293946,-0.178756,-0.281361,-0.396043,...,0.000003,0.000002,0.000008,0.000003,0.000007,0.000022,0.000004,0.000017,0.000002,0.000003


In [6]:
print(len(filtered_df['Participant'].unique()))

23


## Standardization by signal type

In [7]:
scaler = StandardScaler()
IMU_df = filtered_df.iloc[:, 3:75]
EMG_df = filtered_df.iloc[:, 75:91]
IMU_mean = IMU_df.values.mean()
EMG_mean = EMG_df.values.mean()
IMU_sd = IMU_df.values.std()
EMG_sd = EMG_df.values.std()
IMU_scaled = pd.DataFrame((IMU_df - IMU_mean) / IMU_sd)
EMG_scaled = pd.DataFrame((EMG_df - EMG_mean) / EMG_sd)
scaled_signal_df = pd.concat([IMU_scaled, EMG_scaled], axis=1)
print(scaled_signal_df.shape)
scaled_signal_df.head()

(147200, 88)


,IMU1_ax,IMU1_ay,IMU1_az,IMU1_vx,IMU1_vy,IMU1_vz,IMU2_ax,IMU2_ay,IMU2_az,IMU2_vx,...,EMG7,EMG8,EMG9,EMG10,EMG11,EMG12,EMG13,EMG14,EMG15,EMG16
0,0.665145,-1.977439,-0.037531,-0.054903,-0.436683,-0.028574,-0.824769,-1.870729,-0.730140,0.217169,...,-0.277563,-0.278480,-0.236531,0.419390,-0.193355,-0.208323,-0.286637,-0.007593,-0.304684,-0.292680
1,0.653560,-2.025361,-0.031507,-0.019762,-0.432190,-0.093388,-0.852989,-1.906012,-0.750769,0.258858,...,-0.245450,-0.253635,-0.224226,0.196058,-0.070718,-0.084532,-0.257935,0.321491,-0.301261,-0.280903
2,0.689353,-2.026432,-0.015400,0.158311,-0.463186,-0.360271,-0.860947,-1.905916,-0.747222,0.250395,...,-0.233207,-0.247789,-0.192642,-0.085739,-0.181639,-0.156848,-0.243568,0.409278,-0.253220,-0.271569
3,0.687915,-1.998772,-0.015815,0.081760,-0.420697,-0.211836,-0.842027,-1.887364,-0.734550,0.342026,...,-0.231170,-0.255493,-0.140612,-0.151117,-0.191213,-0.220202,-0.210332,0.233681,-0.246135,-0.229901
4,0.728695,-2.083271,0.021992,0.566489,-0.408089,-0.619631,-0.856073,-1.903135,-0.747477,0.351151,...,-0.219437,-0.263912,-0.053564,-0.229170,-0.096632,0.517767,-0.209701,0.321423,-0.285006,-0.235730


## Standardization by Column

In [8]:
scaler = StandardScaler()
scaled_column_df = pd.DataFrame(scaler.fit_transform(filtered_df.iloc[:,3:]), columns=filtered_df.columns[3:91])
print(scaled_column_df.shape)
scaled_column_df.head()

(147200, 88)


,IMU1_ax,IMU1_ay,IMU1_az,IMU1_vx,IMU1_vy,IMU1_vz,IMU2_ax,IMU2_ay,IMU2_az,IMU2_vx,...,EMG7,EMG8,EMG9,EMG10,EMG11,EMG12,EMG13,EMG14,EMG15,EMG16
0,-0.584221,-0.940084,-0.618697,0.068040,0.043047,-0.037757,0.828384,-1.071060,-0.003160,-0.042474,...,-0.290377,-0.384287,-0.470764,0.450200,-0.250753,-0.296238,-0.358187,0.023753,-0.232261,-0.261780
1,-0.611052,-1.058715,-0.609062,0.101518,0.047787,-0.157921,0.772465,-1.144022,-0.036870,0.001795,...,-0.173766,-0.361815,-0.453785,0.192799,-0.134722,-0.219981,-0.310803,0.339430,-0.230559,-0.254007
2,-0.528152,-1.061365,-0.583300,0.271166,0.015083,-0.652714,0.756696,-1.143823,-0.031073,-0.007192,...,-0.129305,-0.356526,-0.410201,-0.131984,-0.239667,-0.264528,-0.287085,0.423641,-0.206672,-0.247846
3,-0.531483,-0.992892,-0.583964,0.198237,0.059914,-0.377520,0.794186,-1.105459,-0.010367,0.090111,...,-0.121908,-0.363494,-0.338401,-0.207335,-0.248726,-0.303555,-0.232217,0.255198,-0.203149,-0.220342
4,-0.437035,-1.202069,-0.523495,0.660031,0.073216,-1.133560,0.766353,-1.138074,-0.031491,0.099801,...,-0.079301,-0.371110,-0.218280,-0.297295,-0.159240,0.151044,-0.231174,0.339365,-0.222477,-0.224189


## Perform PCA

In [9]:
pca = PCA(n_components=3)
pca_by_signal_result_df = pd.DataFrame(data=pca.fit_transform(scaled_signal_df), columns=['PC1', 'PC2', 'PC3'])
pca_by_column_result_df = pd.DataFrame(data=pca.fit_transform(scaled_column_df), columns=['PC1', 'PC2', 'PC3'])
filtered_df = filtered_df.reset_index(drop=True)
pca_by_signal_result_df = pca_by_signal_result_df.reset_index(drop=True)
pca_by_column_result_df = pca_by_column_result_df.reset_index(drop=True)
pca_by_signal_df = pd.concat([filtered_df.iloc[:,0:3], pca_by_signal_result_df], axis=1)
pca_by_column_df = pd.concat([filtered_df.iloc[:,0:3], pca_by_column_result_df], axis=1)
print(pca_by_signal_df.shape)
pca_by_signal_df.head()


(147200, 6)


,Participant,Gesture_ID,Gesture_Num,PC1,PC2,PC3
0,P102,pan,1,-0.191847,-0.700971,-0.870333
1,P102,pan,1,-0.154181,-0.668933,-0.774758
2,P102,pan,1,-0.003461,-0.646763,-0.934882
3,P102,pan,1,-0.204800,-0.610422,-0.941988
4,P102,pan,1,0.158027,-0.621719,-0.394332


## Plot PCA

In [ ]:
# Create the 2D scatter plot
fig = px.scatter(pca_by_column_df, 
                 x='PC1', 
                 y='PC2', 
                 color='Participant', 
                 title='2D PCA of Gesture Data Standardized by Channel, Coloring Based on Participant',
                 )

# Update layout to remove axis numbers and increase plot size
fig.update_layout(
    title=dict(
        font=dict(size=24)
    ), 
    xaxis=dict(showticklabels=False, title='PC1'),
    yaxis=dict(showticklabels=False, title='PC2'),
    width=1200,  # Set the width of the plot
    height=800,  # Set the height of the plot
    margin=dict(l=20, r=20, t=50, b=20),  # Adjust margins
    legend=dict(
        x=0.8,  # Adjust x position of the legend
        y=0.9,  # Adjust y position of the legend
        traceorder='normal',
        font=dict(
            size=16,
        ),
        bgcolor='rgba(255, 255, 255, 0.5)',
        bordercolor='Black',
        borderwidth=1
    )
)

fig.update_traces(marker=dict(size=3))  # Adjust marker size as needed

fig.show()

In [ ]:
# Create the 2D scatter plot
fig = px.scatter(pca_by_column_df, 
                 x='PC1', 
                 y='PC2', 
                 color='Gesture_ID', 
                 title='2D PCA of Gesture Data Standardized by Channel, Coloring Based on Gesture',
                 )

# Update layout to remove axis numbers and increase plot size
fig.update_layout(
    title=dict(
        font=dict(size=24)
    ), 
    xaxis=dict(showticklabels=False, title='PC1'),
    yaxis=dict(showticklabels=False, title='PC2'),
    width=1200,  # Set the width of the plot
    height=800,  # Set the height of the plot
    margin=dict(l=20, r=20, t=50, b=20),  # Adjust margins
    legend=dict(
        x=0.8,  # Adjust x position of the legend
        y=0.9,  # Adjust y position of the legend
        traceorder='normal',
        font=dict(
            size=16,
        ),
        bgcolor='rgba(255, 255, 255, 0.5)',
        bordercolor='Black',
        borderwidth=1
    )
)

fig.update_traces(marker=dict(size=3))  # Adjust marker size as needed

fig.show()

In [ ]:
# Convert Participant and Gesture_ID to categorical codes for color and symbol
# Create the 2D scatter plot
fig = px.scatter(pca_by_signal_df, 
                 x='PC1', 
                 y='PC2', 
                 color='Participant', 
                 title='2D PCA of Gesture Data Standardized by Signal, Coloring Based on Participant',
                 )

# Update layout to remove axis numbers and increase plot size
fig.update_layout(
    title=dict(
        font=dict(size=24)
    ), 
    xaxis=dict(showticklabels=False, title='PC1'),
    yaxis=dict(showticklabels=False, title='PC2'),
    width=1200,  # Set the width of the plot
    height=800,  # Set the height of the plot
    margin=dict(l=20, r=20, t=50, b=20),  # Adjust margins
    legend=dict(
        x=0.8,  # Adjust x position of the legend
        y=0.9,  # Adjust y position of the legend
        traceorder='normal',
        font=dict(
            size=16,
        ),
        bgcolor='rgba(255, 255, 255, 0.5)',
        bordercolor='Black',
        borderwidth=1
    )
)

fig.update_traces(marker=dict(size=3))  # Adjust marker size as needed

fig.show()

In [ ]:
# Convert Participant and Gesture_ID to categorical codes for color and symbol
# Create the 2D scatter plot
fig = px.scatter(pca_by_signal_df, 
                 x='PC1', 
                 y='PC2', 
                 color='Gesture_ID', 
                 title='2D PCA of Gesture Data Standardized by Signal, Coloring Based on Gesture',
                 )

# Update layout to remove axis numbers and increase plot size
fig.update_layout(
    title=dict(
        font=dict(size=24)
    ), 
    xaxis=dict(showticklabels=False, title='PC1'),
    yaxis=dict(showticklabels=False, title='PC2'),
    width=1200,  # Set the width of the plot
    height=800,  # Set the height of the plot
    margin=dict(l=20, r=20, t=50, b=20),  # Adjust margins
    legend=dict(
        x=0.8,  # Adjust x position of the legend
        y=0.9,  # Adjust y position of the legend
        traceorder='normal',
        font=dict(
            size=16,
        ),
        bgcolor='rgba(255, 255, 255, 0.5)',
        bordercolor='Black',
        borderwidth=1
    )
)

fig.update_traces(marker=dict(size=3))  # Adjust marker size as needed

fig.show()

In [ ]:
fig = px.scatter_3d(pca_by_column_df, 
                    x='PC1', 
                    y='PC2', 
                    z='PC3', 
                    color='Gesture_ID', 
                    title='3D PCA of Biosignal Data, Standardized by Channel, Coloring Based on Gesture')

fig.update_traces(marker=dict(size=3))  # Adjust marker size as needed

fig.update_layout(
    title=dict(
        font=dict(size=24)
    ), 
    scene=dict(
        xaxis=dict(showticklabels=False, title='PC1'),
        yaxis=dict(showticklabels=False, title='PC2'),
        zaxis=dict(showticklabels=False, title='PC3')
    ),
    width=1200,  # Set the width of the plot
    height=800,  # Set the height of the plot
    margin=dict(l=20, r=20, t=50, b=20),  # Adjust margins
    legend=dict(
        x=0.8,  # Adjust x position of the legend
        y=0.9,  # Adjust y position of the legend
        traceorder='normal',
        font=dict(
            size=16,
        ),
        bgcolor='rgba(255, 255, 255, 0.5)',
        bordercolor='Black',
        borderwidth=1
    )
)

fig.show()

In [ ]:
fig = px.scatter_3d(pca_by_column_df, 
                    x='PC1', 
                    y='PC2', 
                    z='PC3', 
                    color='Participant', 
                    title='3D PCA of Biosignal Data, Standardized by Channel, Coloring Based on Participant')

fig.update_traces(marker=dict(size=3))  # Adjust marker size as needed
fig.update_layout(
    title=dict(
        font=dict(size=24)
    ), 
    scene=dict(
        xaxis=dict(showticklabels=False, title='PC1'),
        yaxis=dict(showticklabels=False, title='PC2'),
        zaxis=dict(showticklabels=False, title='PC3')
    ),
    width=1200,  # Set the width of the plot
    height=800,  # Set the height of the plot
    margin=dict(l=20, r=20, t=50, b=20),  # Adjust margins
    legend=dict(
        x=0.8,  # Adjust x position of the legend
        y=0.9,  # Adjust y position of the legend
        traceorder='normal',
        font=dict(
            size=16,
        ),
        bgcolor='rgba(255, 255, 255, 0.5)',
        bordercolor='Black',
        borderwidth=1
    )
)

fig.show()

In [ ]:
fig = px.scatter_3d(pca_by_signal_df, 
                    x='PC1', 
                    y='PC2', 
                    z='PC3', 
                    color='Gesture_ID', 
                    title='3D PCA of Biosignal Data, Standardized by Signal, Coloring Based on Gesture')

fig.update_traces(marker=dict(size=3))  # Adjust marker size as needed

fig.update_layout(
    title=dict(
        font=dict(size=24)
    ), 
    scene=dict(
        xaxis=dict(showticklabels=False, title='PC1'),
        yaxis=dict(showticklabels=False, title='PC2'),
        zaxis=dict(showticklabels=False, title='PC3')
    ),
    width=1200,  # Set the width of the plot
    height=800,  # Set the height of the plot
    margin=dict(l=20, r=20, t=50, b=20),  # Adjust margins
    legend=dict(
        x=0.8,  # Adjust x position of the legend
        y=0.9,  # Adjust y position of the legend
        traceorder='normal',
        font=dict(
            size=16,
        ),
        bgcolor='rgba(255, 255, 255, 0.5)',
        bordercolor='Black',
        borderwidth=1
    )
)

fig.show()

In [ ]:
fig = px.scatter_3d(pca_by_signal_df, 
                    x='PC1', 
                    y='PC2', 
                    z='PC3', 
                    color='Participant', 
                    title='3D PCA of Biosignal Data, Standardized by Signal, Coloring Based on Participant')

fig.update_traces(marker=dict(size=3))  # Adjust marker size as needed
fig.update_layout(
    title=dict(
        font=dict(size=24)
    ), 
    scene=dict(
        xaxis=dict(showticklabels=False, title='PC1'),
        yaxis=dict(showticklabels=False, title='PC2'),
        zaxis=dict(showticklabels=False, title='PC3')
    ),
    width=1200,  # Set the width of the plot
    height=800,  # Set the height of the plot
    margin=dict(l=20, r=20, t=50, b=20),  # Adjust margins
    legend=dict(
        x=0.8,  # Adjust x position of the legend
        y=0.9,  # Adjust y position of the legend
        traceorder='normal',
        font=dict(
            size=16,
        ),
        bgcolor='rgba(255, 255, 255, 0.5)',
        bordercolor='Black',
        borderwidth=1
    )
)

fig.show()

## Add Handedness 

In [16]:
import warnings
warnings.filterwarnings('ignore')

In [17]:
# Read all sheets into a dictionary of dataframes
all_sheets_dict = pd.read_excel("C:\\Users\\rubin\\Research\\user_gesture_descriptions_qualitative_analysis.xlsx", sheet_name=None)

# Initialize an empty list to hold individual dataframes
df_list = []

# Iterate through the dictionary and process each sheet
for user_id, df in all_sheets_dict.items():
    df.columns.values[1] = 'gesture'
    # Identify columns with "Unnamed" in their names
    unnamed_cols = df.columns.str.contains("Unnamed")
    # Drop those columns
    df = df.loc[:, ~unnamed_cols]
    df = df.iloc[:10,:]
    df= df.reset_index(drop=True)
    df_list.append(df)  # Add the dataframe to the list

In [18]:
print(df_list[2].shape)
df_list[2].head()

(10, 17)


,PID,gesture,time,description,hand used,lifted arms off arm rest?,dynamic gesture?,muscles activated 1,muscles activated 2,muscles activated 3,agreement,midair or on table,gesture ideas origin 1,gesture ideas origin 2,gesture ideas origin 3,one hand v 2 hands,one hand v 2 hands.1
0,P103,rotate,3,make a circle with right hand without moving f...,right,barely,yes,upper-body (bicep/tricep),shoulder,NaN,rotate from shoulders,midair,what I can do with my abilities,NaN,NaN,due to handedness,NaN
1,P103,move,2,move right hand from left to right without rai...,right,barely,yes,upper-body (bicep/tricep),NaN,NaN,point and move entire hand in 2D (like on a fl...,on a 2D plane but not on table,NaN,NaN,NaN,NaN,NaN
2,P103,select-single,2,using a mouse to select,right,barely,yes,shoulder,upper-body (bicep/tricep),NaN,point at table (like clicking a mouse),on a 2D plane but not on table,NaN,NaN,NaN,NaN,NaN
3,P103,zoom-in,3,move head forward and backwards,head,no,yes,shoulder,NaN,NaN,move head closer/away to screen,neither,NaN,NaN,NaN,NaN,NaN
4,P103,zoom-out,3,"move head backwards, then forward",head,no,yes,shoulder,NaN,NaN,move head closer/away to screen,neither,NaN,NaN,NaN,NaN,NaN


In [19]:
# Concatenate all DataFrames in the list into one large DataFrame
all_PID_xlsx_df = pd.concat(df_list[1:], ignore_index=True)
# Optionally, reset the index if needed
all_PID_xlsx_df.reset_index(drop=True, inplace=True)
all_PID_xlsx_df.rename(columns={'PID': 'Participant', 'gesture': 'Gesture_ID', 'hand used': 'Handedness', 'lifted arms off arm rest?': 'Lifted Arms' }, inplace=True)
print(all_PID_xlsx_df.shape)
all_PID_xlsx_df.head()

(270, 22)


,Participant,Gesture_ID,time,description,Handedness,Lifted Arms,dynamic gesture?,muscles activated 1,muscles activated 2,muscles activated 3,...,gesture ideas origin 1,gesture ideas origin 2,gesture ideas origin 3,one hand v 2 hands,one hand v 2 hands.1,notes,muscles activated 4,body part used 1,body part used 2,body part used 3
0,P102,rotate,4,rotate entire right hand from wrist,right,yes,yes,wrist/forearm,shoulder,fingers,...,tablet/touchscreen,NaN,NaN,due to handedness,other hand is too disabled,NaN,NaN,NaN,NaN,NaN
1,P102,move,2,point right hand and move entire arm from left...,right,yes,yes,shoulder,upper-body (bicep/tricep),fingers,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,P102,select-single,3,point at object using right index finger,right,yes,yes,shoulder,upper-body (bicep/tricep),fingers,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,P102,zoom-in,2,use two hands to make a frame and move hands c...,both,yes,yes,shoulder,upper-body (bicep/tricep),fingers,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,P102,zoom-out,5,use right thumb and index finger and pinch fin...,right,yes,yes,fingers,wrist/forearm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
filtered_metadata_df = all_PID_xlsx_df.loc[:,['Participant','Gesture_ID','Handedness', 'Lifted Arms','muscles activated 1']]
print(filtered_metadata_df.shape)
# make sure same users are in both dfs
filtered_metadata_df = filtered_metadata_df[filtered_metadata_df['Participant'].isin(['P102','P103', 'P104', 'P105', 'P106', 'P107', 'P108', 'P109', 'P110', 'P112', 'P114', 'P115', 'P116', 'P118', 'P119', 'P121', 'P122', 'P123', 'P125', 'P126',
 'P127', 'P128', 'P132'])]
print(filtered_metadata_df.shape)
print(set(pca_by_signal_df['Participant'].unique()) == set(filtered_metadata_df['Participant'].unique()))

(270, 5)
(230, 5)
True


In [21]:
pca_signal_meta_df = pd.merge(filtered_metadata_df, pca_by_signal_df, on=['Participant', 'Gesture_ID'], how='inner')
pca_column_meta_df = pd.merge(filtered_metadata_df, pca_by_column_df, on=['Participant', 'Gesture_ID'], how='inner')
print(pca_signal_meta_df.shape)
pca_signal_meta_df.head()


(147200, 9)


,Participant,Gesture_ID,Handedness,Lifted Arms,muscles activated 1,Gesture_Num,PC1,PC2,PC3
0,P102,rotate,right,yes,wrist/forearm,1,-0.239026,-0.611281,-0.876869
1,P102,rotate,right,yes,wrist/forearm,1,-0.249440,-0.677231,-0.860271
2,P102,rotate,right,yes,wrist/forearm,1,-0.149920,-0.656097,-0.852871
3,P102,rotate,right,yes,wrist/forearm,1,-0.202984,-0.613159,-0.785642
4,P102,rotate,right,yes,wrist/forearm,1,0.551187,-0.709154,-0.579720


## Make sure labeling is consistent

In [23]:
print(pca_signal_meta_df['Handedness'].unique())
print(pca_signal_meta_df['Lifted Arms'].unique())
print(pca_signal_meta_df['muscles activated 1'].unique())

['right' 'both' 'head' 'left' 'shoulder']
['yes' 'barely' 'no']
['wrist/forearm' 'shoulder' 'fingers' 'upper-body (bicep/tricep)']


## plot based on handedness

In [ ]:
fig = px.scatter_3d(pca_signal_meta_df, 
                    x='PC1', 
                    y='PC2', 
                    z='PC3', 
                    color='Handedness',
                    title='3D PCA of Biosignal Data, Standardized by Signal, Handedness')

# Adjust marker size for the scatter plot and hide the legend for these traces
fig.update_traces(marker=dict(size=3), showlegend=False)

# Get unique Handedness values and their corresponding colors
unique_handedness = pca_signal_meta_df['Handedness'].unique()
color_map = px.colors.qualitative.Plotly  # Use the same color scheme as Plotly

# Add invisible traces with larger markers for the legend
for i, handedness in enumerate(unique_handedness):
    fig.add_scatter3d(
        x=[None], y=[None], z=[None],
        mode='markers',
        marker=dict(size=15, color=color_map[i % len(color_map)]),  # Larger marker size for the legend
        legendgroup=handedness,
        showlegend=True,
        name=handedness
    )

# Update layout
fig.update_layout(
    title=dict(
        font=dict(size=24)
    ), 
    scene=dict(
        xaxis=dict(showticklabels=False, title='PC1'),
        yaxis=dict(showticklabels=False, title='PC2'),
        zaxis=dict(showticklabels=False, title='PC3')
    ),
    width=1200,  # Set the width of the plot
    height=800,  # Set the height of the plot
    margin=dict(l=20, r=20, t=50, b=20),  # Adjust margins
    legend=dict(
        x=0.8,  # Adjust x position of the legend
        y=0.9,  # Adjust y position of the legend
        traceorder='normal',
        font=dict(
            size=16,
        ),
        bgcolor='rgba(255, 255, 255, 0.5)',
        bordercolor='Black',
        borderwidth=1
    )
)

fig.show()

In [ ]:
fig = px.scatter_3d(pca_column_meta_df, 
                    x='PC1', 
                    y='PC2', 
                    z='PC3', 
                    color='Handedness',
                    title='3D PCA of Biosignal Data, Standardized by Channel, Handedness')

# Adjust marker size for the scatter plot and hide the legend for these traces
fig.update_traces(marker=dict(size=3), showlegend=False)

# Get unique Handedness values and their corresponding colors
unique_handedness = pca_signal_meta_df['Handedness'].unique()
color_map = px.colors.qualitative.Plotly  # Use the same color scheme as Plotly

# Add invisible traces with larger markers for the legend
for i, handedness in enumerate(unique_handedness):
    fig.add_scatter3d(
        x=[None], y=[None], z=[None],
        mode='markers',
        marker=dict(size=15, color=color_map[i % len(color_map)]),  # Larger marker size for the legend
        legendgroup=handedness,
        showlegend=True,
        name=handedness
    )

# Update layout
fig.update_layout(
    title=dict(
        font=dict(size=24)
    ), 
    scene=dict(
        xaxis=dict(showticklabels=False, title='PC1'),
        yaxis=dict(showticklabels=False, title='PC2'),
        zaxis=dict(showticklabels=False, title='PC3')
    ),
    width=1200,  # Set the width of the plot
    height=800,  # Set the height of the plot
    margin=dict(l=20, r=20, t=50, b=20),  # Adjust margins
    legend=dict(
        x=0.8,  # Adjust x position of the legend
        y=0.9,  # Adjust y position of the legend
        traceorder='normal',
        font=dict(
            size=16,
        ),
        bgcolor='rgba(255, 255, 255, 0.5)',
        bordercolor='Black',
        borderwidth=1
    )
)

fig.show()

## plot Based on Lifted Arms

In [ ]:
fig = px.scatter_3d(pca_signal_meta_df, 
                    x='PC1', 
                    y='PC2', 
                    z='PC3', 
                    color='Lifted Arms',
                    title='3D PCA of Biosignal Data, Standardized by Signal, Lifting Arms')

# Adjust marker size for the scatter plot and hide the legend for these traces
fig.update_traces(marker=dict(size=3), showlegend=False)

# Get unique Handedness values and their corresponding colors
unique_handedness = pca_signal_meta_df['Lifted Arms'].unique()
color_map = px.colors.qualitative.Plotly  # Use the same color scheme as Plotly

# Add invisible traces with larger markers for the legend
# I couldnt be bothered to change the var names
for i, handedness in enumerate(unique_handedness):
    fig.add_scatter3d(
        x=[None], y=[None], z=[None],
        mode='markers',
        marker=dict(size=15, color=color_map[i % len(color_map)]),  # Larger marker size for the legend
        legendgroup=handedness,
        showlegend=True,
        name=handedness
    )

# Update layout
fig.update_layout(
    title=dict(
        font=dict(size=24)
    ), 
    scene=dict(
        xaxis=dict(showticklabels=False, title='PC1'),
        yaxis=dict(showticklabels=False, title='PC2'),
        zaxis=dict(showticklabels=False, title='PC3')
    ),
    width=1200,  # Set the width of the plot
    height=800,  # Set the height of the plot
    margin=dict(l=20, r=20, t=50, b=20),  # Adjust margins
    legend=dict(
        x=0.8,  # Adjust x position of the legend
        y=0.9,  # Adjust y position of the legend
        traceorder='normal',
        font=dict(
            size=16,
        ),
        bgcolor='rgba(255, 255, 255, 0.5)',
        bordercolor='Black',
        borderwidth=1
    )
)

fig.show()

In [ ]:
fig = px.scatter_3d(pca_column_meta_df, 
                    x='PC1', 
                    y='PC2', 
                    z='PC3', 
                    color='Lifted Arms',
                    title='3D PCA of Biosignal Data, Standardized by Channel, Lifting Arms')

# Adjust marker size for the scatter plot and hide the legend for these traces
fig.update_traces(marker=dict(size=3), showlegend=False)

# Get unique Handedness values and their corresponding colors
unique_handedness = pca_signal_meta_df['Lifted Arms'].unique()
color_map = px.colors.qualitative.Plotly  # Use the same color scheme as Plotly

# Add invisible traces with larger markers for the legend
# I couldnt be bothered to change the var names
for i, handedness in enumerate(unique_handedness):
    fig.add_scatter3d(
        x=[None], y=[None], z=[None],
        mode='markers',
        marker=dict(size=15, color=color_map[i % len(color_map)]),  # Larger marker size for the legend
        legendgroup=handedness,
        showlegend=True,
        name=handedness
    )

# Update layout
fig.update_layout(
    title=dict(
        font=dict(size=24)
    ), 
    scene=dict(
        xaxis=dict(showticklabels=False, title='PC1'),
        yaxis=dict(showticklabels=False, title='PC2'),
        zaxis=dict(showticklabels=False, title='PC3')
    ),
    width=1200,  # Set the width of the plot
    height=800,  # Set the height of the plot
    margin=dict(l=20, r=20, t=50, b=20),  # Adjust margins
    legend=dict(
        x=0.8,  # Adjust x position of the legend
        y=0.9,  # Adjust y position of the legend
        traceorder='normal',
        font=dict(
            size=16,
        ),
        bgcolor='rgba(255, 255, 255, 0.5)',
        bordercolor='Black',
        borderwidth=1
    )
)

fig.show()

## Plot based on primary muscles activated

In [ ]:
fig = px.scatter_3d(pca_signal_meta_df, 
                    x='PC1', 
                    y='PC2', 
                    z='PC3', 
                    color='muscles activated 1',
                    title='3D PCA of Biosignal Data, Standardized by Signal, Primary Muscles Activated')

# Adjust marker size for the scatter plot and hide the legend for these traces
fig.update_traces(marker=dict(size=3), showlegend=False)

# Get unique Handedness values and their corresponding colors
unique_handedness = pca_signal_meta_df['muscles activated 1'].unique()
color_map = px.colors.qualitative.Plotly  # Use the same color scheme as Plotly

# Add invisible traces with larger markers for the legend
# I couldnt be bothered to change the var names
for i, handedness in enumerate(unique_handedness):
    fig.add_scatter3d(
        x=[None], y=[None], z=[None],
        mode='markers',
        marker=dict(size=15, color=color_map[i % len(color_map)]),  # Larger marker size for the legend
        legendgroup=handedness,
        showlegend=True,
        name=handedness
    )

# Update layout
fig.update_layout(
    title=dict(
        font=dict(size=24)
    ), 
    scene=dict(
        xaxis=dict(showticklabels=False, title='PC1'),
        yaxis=dict(showticklabels=False, title='PC2'),
        zaxis=dict(showticklabels=False, title='PC3')
    ),
    width=1200,  # Set the width of the plot
    height=800,  # Set the height of the plot
    margin=dict(l=20, r=20, t=50, b=20),  # Adjust margins
    legend=dict(
        title=dict(text='Primary Muscles Activated'),
        x=0.7,  # Adjust x position of the legend
        y=0.9,  # Adjust y position of the legend
        traceorder='normal',
        font=dict(
            size=16,
        ),
        bgcolor='rgba(255, 255, 255, 0.5)',
        bordercolor='Black',
        borderwidth=1
    )
)

fig.show()

In [ ]:
fig = px.scatter_3d(pca_column_meta_df, 
                    x='PC1', 
                    y='PC2', 
                    z='PC3', 
                    color='muscles activated 1',
                    title='3D PCA of Biosignal Data, Standardized by Channel, Primary Muscles Activated')

# Adjust marker size for the scatter plot and hide the legend for these traces
fig.update_traces(marker=dict(size=3), showlegend=False)

# Get unique Handedness values and their corresponding colors
unique_handedness = pca_signal_meta_df['muscles activated 1'].unique()
color_map = px.colors.qualitative.Plotly  # Use the same color scheme as Plotly

# Add invisible traces with larger markers for the legend
# I couldnt be bothered to change the var names
for i, handedness in enumerate(unique_handedness):
    fig.add_scatter3d(
        x=[None], y=[None], z=[None],
        mode='markers',
        marker=dict(size=15, color=color_map[i % len(color_map)]),  # Larger marker size for the legend
        legendgroup=handedness,
        showlegend=True,
        name=handedness
    )

# Update layout
fig.update_layout(
    title=dict(
        font=dict(size=24)
    ), 
    scene=dict(
        xaxis=dict(showticklabels=False, title='PC1'),
        yaxis=dict(showticklabels=False, title='PC2'),
        zaxis=dict(showticklabels=False, title='PC3')
    ),
    width=1200,  # Set the width of the plot
    height=800,  # Set the height of the plot
    margin=dict(l=20, r=20, t=50, b=20),  # Adjust margins
    legend=dict(
        title=dict(text='Primary Muscles Activated'),
        x=0.7,  # Adjust x position of the legend
        y=0.9,  # Adjust y position of the legend
        traceorder='normal',
        font=dict(
            size=16,
        ),
        bgcolor='rgba(255, 255, 255, 0.5)',
        bordercolor='Black',
        borderwidth=1
    )
)

fig.show()